In [1]:
import requests
import csv
import pandas as pd

In [2]:
def get_id(api_key):
    '''This function takes an argument- the api key, and return the typeform_uid 
    that will be used to access the document associated with that uid'''
    global u_id
    
    url1='https://api.typeform.com/v1/forms?&key='+api_key
    resp = requests.get(url1)
    if(resp.ok):
        for item in resp.json():
            print(resp.json())
            u_id = item['id']
    else:
        resp.raise_for_status()

In [3]:
def get_documents(uid, api_key):
    '''This function takes two arguments-uid and the api key, and produces a JSON document'''
    global respJSON
    url2='https://api.typeform.com/v1/form/'+uid+'?key='+api_key
    resp2 = requests.get(url2)
    if(resp2.ok):
        respJSON = resp2.json()
        return respJSON 
    else:
        resp2.raise_for_status()

In [4]:
#call both functions

key='ac83034cfa742c0f79c26e9a612b4ba7e2aa0d3d'
get_id(key)
get_documents(u_id,key)

[{'id': 'PfSNFM', 'name': 'Keboola Test Dungeon'}]


{'http_status': 200,
 'questions': [{'field_id': 46098446,
   'id': 'date_46098446',
   'question': 'This is a date field. Say "Hello, date field!" So, what date is today?'},
  {'field_id': 46099137,
   'id': 'number_46099137',
   'question': 'This is a number field. It contains\xa0numbers. What is your favorite prime number? #GeekAllert'},
  {'field_id': 46099317,
   'id': 'yesno_46099317',
   'question': 'We went rogue and added the checkmark field! Is Keboola the best company in the known universe to work for?'},
  {'field_id': 46100308,
   'id': 'list_46100308_choice',
   'question': 'Now is time for the multi-choice question! What is your favorite kind of Pho?'},
  {'field_id': 46100513,
   'id': 'textarea_46100513',
   'question': 'The dirtiest data field of all data fields. The address field. The true nightmare of every data analyst!'}],
 'responses': [{'answers': {'date_46098446': '2017-03-17',
    'list_46100308_choice': 'I love all kinds of Pho. Pho for the president!',
    '

In [5]:
#Get a header for csv
#I believe the useful header (column) will be the answers (response to the questions) 
#as well as the date submitted and network id

#Here we get the key of network_id and date_submit as strings
metadata_keys = list(respJSON['responses'][0]['metadata'].keys())

for i in metadata_keys:
    if i == 'network_id':
        header_1 = i
        
    if i == 'date_submit':
        header_2 = i
        
#Here, we get the id of each question and use it as a header
#I am only using the id to keep the header looking clean
#we will also use this variable later to make sure our responses matches the header
        
questions_id = []
for i in range(len(respJSON['questions'])):
    questions_id.append(respJSON['questions'][i]['id'])
        
#create the header

header = [str(header_1), str(header_2)]
for i in range(len(questions_id)):
    header.append(str(questions_id[i]))
    


In [10]:
#Get answers from responses matching the question id in the header

all_rows = [[] for _ in range(len(respJSON['responses']))]

for i in range(len(respJSON['responses'])):
    rows=[]
    
    try:
        rows.append(respJSON['responses'][i]['metadata']['network_id'])
    except:
        rows.append('NaN')
    try:
        rows.append(respJSON['responses'][i]['metadata']['date_submit'])
    except:
        rows.append('NaN')
    
    for j in range(len(questions_id)):
        try:
            rows.append(respJSON['responses'][i]['answers'][questions_id[j]].replace('\n', ' '))
        except:
            rows.append('NaN')
    
    all_rows[i].extend(rows)

In [11]:
#write to CSV

with open('documents.csv', 'w', newline='') as documents:
    csvwriter = csv.writer(documents)
    
    csvwriter.writerow(header)
    
    for row in all_rows:
        csvwriter.writerow(row) 
    
documents.close()

In [12]:
with open('documents.csv', 'r') as doc:
    for row in doc:
        print(row)

network_id,date_submit,date_46098446,number_46099137,yesno_46099317,list_46100308_choice,textarea_46100513

783db4e9d8,2017-03-17 21:23:50,2017-03-17,3,1,I love all kinds of Pho. Pho for the president!,"210 - 1461 Johnston Rd, White Rock, BC V4B 3Z4"

783db4e9d8,2017-03-17 21:25:08,2016-12-31,7,NaN,"More ingredients, the better. I like my Pho full of taste. ",15230 Russell Ave White Rock BC V4B

783db4e9d8,0000-00-00 00:00:00,NaN,NaN,NaN,NaN,NaN

783db4e9d8,2017-03-17 21:27:06,1921-04-01,13,1,"I don't eat Pho but I'll start immediately! So sorry, so Pho!","Krizikova 488/115, 186 00 Praha 8-Karlin, Czechia"

783db4e9d8,2017-03-17 21:30:07,2007-01-01,10,1,Beef and basic is my choice of Pho.,"30 Cecil St, #19-06 Singapore 049712"

783db4e9d8,2017-03-17 21:32:34,2015-02-16,17,1,NaN,"WeWork 9 Devonshire Square, London EC2M 4YD, United Kingdom"

783db4e9d8,2017-03-17 21:34:04,NaN,101,NaN,I love all kinds of Pho. Pho for the president!,"1150 Davie St, Vancouver, BC V6E 1N1"

e367c5e34e,2017-0

In [16]:
df = pd.read_csv('documents.csv', sep=',')
df

,network_id,date_submit,date_46098446,number_46099137,yesno_46099317,list_46100308_choice,textarea_46100513
0,783db4e9d8,2017-03-17 21:23:50,2017-03-17,3.0,1.0,I love all kinds of Pho. Pho for the president!,"210 - 1461 Johnston Rd, White Rock, BC V4B 3Z4"
1,783db4e9d8,2017-03-17 21:25:08,2016-12-31,7.0,NaN,"More ingredients, the better. I like my Pho fu...",15230 Russell Ave White Rock BC V4B
2,783db4e9d8,0000-00-00 00:00:00,NaN,NaN,NaN,NaN,NaN
3,783db4e9d8,2017-03-17 21:27:06,1921-04-01,13.0,1.0,I don't eat Pho but I'll start immediately! So...,"Krizikova 488/115, 186 00 Praha 8-Karlin, Czechia"
4,783db4e9d8,2017-03-17 21:30:07,2007-01-01,10.0,1.0,Beef and basic is my choice of Pho.,"30 Cecil St, #19-06 Singapore 049712"
5,783db4e9d8,2017-03-17 21:32:34,2015-02-16,17.0,1.0,NaN,"WeWork 9 Devonshire Square, London EC2M 4YD, U..."
6,783db4e9d8,2017-03-17 21:34:04,NaN,101.0,NaN,I love all kinds of Pho. Pho for the president!,"1150 Davie St, Vancouver, BC V6E 1N1"
7,e367c5e34e,2017-03-23 19:37:34,2017-03-23,7.0,1.0,Beef and basic is my choice of Pho.,"123 Evergreen Terrace Springfield, IL 12345"
8,e367c5e34e,2017-03-23 19:38:23,2017-03-23,1.0,NaN,NaN,"Left the multi-choice blank, just because"
9,e367c5e34e,2017-03-23 19:41:22,2017-03-23,7.0,1.0,Beef and basic is my choice of Pho.,"123 Evergreen Terrace Springfield, IL 12345"
